# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
\mu = \int r ρ(r) dr
$$
with respect to a small uniform electric field ``E = -x``.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of ``a`` bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis, tol=tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770400872171                   -0.53    8.0
  2   -2.771687546625       -2.89       -1.32    1.0
  3   -2.771714348282       -4.57       -2.52    1.0
  4   -2.771714676817       -6.48       -3.31    2.0
  5   -2.771714711513       -7.46       -3.73    2.0
┌ Warning: Negative ρ detected
│   min_ρ = -4.8064731693086654e-18
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:7
  6   -2.771714715167       -8.44       -4.69    2.0


-0.00013535817148517545

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε, tol=tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770329832315                   -0.53    8.0
  2   -2.771770441369       -2.84       -1.30    1.0
  3   -2.771801980427       -4.50       -2.66    2.0
  4   -2.771802073720       -7.03       -4.17    2.0
  5   -2.771802074447       -9.14       -4.74    3.0


0.017613101319880697

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013535817148517545
Displaced dipole:  0.017613101319880697
Polarizability :   1.7748459491365873


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, ``\chi_0`` is the
independent-particle polarizability, and ``K`` the
Hartree-exchange-correlation kernel. We denote with ``\delta V_{\rm ext}`` an external
perturbing potential (like in this case the uniform electric field). Then:
$$
\delta\rho = \chi_0 \delta V = \chi_0 (\delta V_{\rm ext} + K \delta\rho),
$$
which implies
$$
\delta\rho = (1-\chi_0 K)^{-1} \chi_0 \delta V_{\rm ext}.
$$
From this we identify the polarizability operator to be ``\chi = (1-\chi_0 K)^{-1} \chi_0``.
Numerically, we apply ``\chi`` to ``\delta V = -x`` by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply (1- χ0 K)
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; ρ=res.ρ)
    χ0δV = apply_χ0(res.ham, res.ψ, res.εF, res.eigenvalues, δV)
    δρ - χ0δV
end

# δVext is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply χ0 once to get non-interacting dipole
δρ_nointeract = apply_χ0(res.ham, res.ψ, res.εF, res.eigenvalues, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

┌ Warning: Negative ρ detected
│   min_ρ = Dual{ForwardDiff.Tag{DFTK.var"#406#409"{PlaneWaveBasis{Float64}, Vector{Matrix{ComplexF64}}, Vector{Matrix{ComplexF64}}, Vector{Vector{Float64}}, Vector{Vector{Float64}}}, Float64}}(-4.8064731693086654e-18,5.572580805322332e-16)
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:7
┌ Warning: Negative ρ detected
│   min_ρ = Dual{ForwardDiff.Tag{DFTK.var"#406#409"{PlaneWaveBasis{Float64}, Vector{Matrix{ComplexF64}}, Vector{Matrix{ComplexF64}}, Vector{Vector{Float64}}, Vector{Vector{Float64}}}, Float64}}(-4.8064731693086654e-18,4.0993440693934737e-16)
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:7
┌ Info: GMRES linsolve in iter 1; step 1: normres = 2.489827097251e-01
└ @ KrylovKit /home/runner/.julia/packages/KrylovKit/kWdb6/src/linsolve/gmres.jl:55
┌ Warning: Negative ρ detected
│   min_ρ = Dual{ForwardDiff.Tag{DFTK.var"#406#409"{PlaneWaveBasis{Float64}, Vector{Matrix{ComplexF64}}, Vector{Matrix{ComplexF64}}, Vector{Vecto

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.